In [1]:
from glob import glob
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import json
import os
import mp
import re
import subprocess
import soundfile as sf

In [2]:
files = glob('klasik_processed/**/*/*.json', recursive = True)

len(files)

1089

In [3]:
rejected = [
    'terima kasih kerana menonton',
    'terima kasih',
    'thank you for watching',
]

In [5]:
# !rm -rf classic-malay-chunk
!mkdir classic-malay-chunk

mkdir: cannot create directory ‘classic-malay-chunk’: File exists


In [6]:
def new_path(f):
    f = f.replace('.mp3', '.alignment')
    f = f.replace('_processed/', '_processed_alignment/')
    return f

def new_path_lang(f):
    f = f.replace('.mp3', '.language')
    f = f.replace('_processed/', '_processed_language/')
    return f

def chunk(alignment, reject = -15, minimum_length = 3.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                temp[-1]['end'] = a['start']
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            alls.append(temp)
    return alls
        
def loop(files):
    files, _ = files
    data = []
    for file in tqdm(files):
        folder = os.path.split(file)[0]
        folder_folder = os.path.split(folder)[1]
        filename = file.replace('.json', '')

        try:
            with open(file) as fopen:
                d = json.load(fopen)
        except:
            continue

        for no, obj in enumerate(d):
            text = obj["text"].strip()
            
            rt_ = re.sub('[^a-z ]+', '', text.lower()).strip()
            if any([s == rt_ for s in rejected]):
                continue
            
            try:
                dense = CountVectorizer(ngram_range = (3,3)).fit_transform([text]).todense()
                repeat = (dense > 3).sum() >= 1
                if repeat:
                    continue
            except:
                continue
            
            audio_path = os.path.join(folder, f'{folder_folder}_{no}.mp3')
            
            if not os.path.exists(audio_path):
                continue
                
            align_path = new_path(audio_path)
            
            if not os.path.exists(align_path):
                continue
                
            with open(align_path) as fopen:
                align = json.load(fopen)
                
            scores = [a for a in align if a['score'] <= -15]
            if not len(scores):
                continue
            
            chunks = chunk(align)
            if len(chunks):
                y, sr = sf.read(audio_path)
                for no, c in enumerate(chunks):
                    try:
                        t = ' '.join([c_['text'] for c_ in c])
                        start = c[0]['start']
                        end = c[-1]['end']
                        a = audio_path.replace('/', '_').replace('.mp3', '') 
                        a = os.path.join('classic-malay-chunk', f'{a}_{no}.mp3')
                        if not os.path.exists(a):
                            sf.write(a, y[int(sr * start): int(sr * end)], sr)

                        data.append({
                            'audio': a,
                            'transcription': t,
                        })
                    except:
                        pass
    
    return data

In [7]:
d = loop((files[-100:], 0))

100%|███████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:20<00:00,  4.81it/s]


In [8]:
len(d)

345

In [9]:
d[-1]

{'audio': 'classic-malay-chunk/klasik_processed_KOTA KUALA KEDAH ~ BENTENG PERTAHANAN MELAYU TERTUA [nfQR9iw6RhE]_KOTA KUALA KEDAH ~ BENTENG PERTAHANAN MELAYU TERTUA [nfQR9iw6RhE]_16_0.mp3',
 'transcription': 'Namun, kota ini juga tidak dapat lari daripada kesan perang apabila musnah diserang musuh sebelum ia dibina semula pada tahun 1711 dan diteruskan kerja-kerja membaiki dan menaik taraf semasa era Sultan'}

In [10]:
import IPython.display as ipd
ipd.Audio(d[-1]['audio'])

In [11]:
data = mp.multiprocessing(files, loop, cores = 15)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:31<00:00,  2.31it/s]


In [15]:
len(data)

3095

In [13]:
from collections import defaultdict

uniques = defaultdict(int)
for d in tqdm(data):
    uniques[d['audio'].split('_processed')[0]] += 1
    
uniques

100%|████████████████████████████████████████████████████████████████████████████████| 3095/3095 [00:00<00:00, 1758516.78it/s]


defaultdict(int, {'classic-malay-chunk/klasik': 3095})

In [14]:
import pandas as pd

df = pd.DataFrame(data)
df.head()

,audio,transcription
0,classic-malay-chunk/klasik_processed_Azzam Sha...,"Okay, saya Azam Syam. Di sini saya nak buat pr..."
1,classic-malay-chunk/klasik_processed_Azzam Sha...,saya yang nak daftar jadi ahli klub Azam Big F...
2,classic-malay-chunk/klasik_processed_Azzam Sha...,"Azam. Uh, betul tak? Uh, tu, saya bukan guru B..."
3,classic-malay-chunk/klasik_processed_Azzam Sha...,"Kita akan dapat kad. Ini contoh sahaja. Okay, ..."
4,classic-malay-chunk/klasik_processed_Azzam Sha...,"alamat saya, aa, juga, aa, nombor IC anda."


In [15]:
df.to_parquet('verify-text-chunk-classic-malay.parquet')

In [17]:
!du -hs classic-malay-chunk

132M	classic-malay-chunk


In [16]:
!zip -rq classic-malay-chunk.zip classic-malay-chunk

In [18]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="verify-text-chunk-classic-malay.parquet",
    path_in_repo="data/classic_malay_chunk-00000-of-00001.parquet",
    repo_id="mesolitica/Malaysian-Voice-Conversion",
    repo_type="dataset",
)


verify-text-chunk-classic-malay.parquet:   0%|          | 0.00/271k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion/commit/58991edac541e0a37c79def520cc0fce79abb754', commit_message='Upload data/classic_malay_chunk-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='58991edac541e0a37c79def520cc0fce79abb754', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Voice-Conversion'), pr_revision=None, pr_num=None)

In [19]:
api.upload_file(
    path_or_fileobj="classic-malay-chunk.zip",
    path_in_repo="data/classic-malay-chunk.zip",
    repo_id="mesolitica/Malaysian-Voice-Conversion",
    repo_type="dataset",
)

classic-malay-chunk.zip:   0%|          | 0.00/130M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion/commit/66c0a6d2f6ae4d8a792e0fc5ca202cebd9c2d91a', commit_message='Upload data/classic-malay-chunk.zip with huggingface_hub', commit_description='', oid='66c0a6d2f6ae4d8a792e0fc5ca202cebd9c2d91a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-Voice-Conversion', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-Voice-Conversion'), pr_revision=None, pr_num=None)